In [0]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer, StandardScaler, VectorAssembler, VectorSlicer
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.functions import vector_to_array
from pyspark.sql.functions import col

In [0]:
train_df = spark.read.csv('/FileStore/tables/train_features.csv', header=True, inferSchema=True)
target_df = spark.read.csv('/FileStore/tables/train_targets_scored.csv', header=True, inferSchema=True)

In [0]:
train_df.persist()
target_df.persist()

Out[2]: DataFrame[sig_id: string, 5-alpha_reductase_inhibitor: int, 11-beta-hsd1_inhibitor: int, acat_inhibitor: int, acetylcholine_receptor_agonist: int, acetylcholine_receptor_antagonist: int, acetylcholinesterase_inhibitor: int, adenosine_receptor_agonist: int, adenosine_receptor_antagonist: int, adenylyl_cyclase_activator: int, adrenergic_receptor_agonist: int, adrenergic_receptor_antagonist: int, akt_inhibitor: int, aldehyde_dehydrogenase_inhibitor: int, alk_inhibitor: int, ampk_activator: int, analgesic: int, androgen_receptor_agonist: int, androgen_receptor_antagonist: int, anesthetic_-_local: int, angiogenesis_inhibitor: int, angiotensin_receptor_antagonist: int, anti-inflammatory: int, antiarrhythmic: int, antibiotic: int, anticonvulsant: int, antifungal: int, antihistamine: int, antimalarial: int, antioxidant: int, antiprotozoal: int, antiviral: int, apoptosis_stimulant: int, aromatase_inhibitor: int, atm_kinase_inhibitor: int, atp-sensitive_potassium_channel_antagonist: int, atp_synthase_inhibitor: int, atpase_inhibitor: int, atr_kinase_inhibitor: int, aurora_kinase_inhibitor: int, autotaxin_inhibitor: int, bacterial_30s_ribosomal_subunit_inhibitor: int, bacterial_50s_ribosomal_subunit_inhibitor: int, bacterial_antifolate: int, bacterial_cell_wall_synthesis_inhibitor: int, bacterial_dna_gyrase_inhibitor: int, bacterial_dna_inhibitor: int, bacterial_membrane_integrity_inhibitor: int, bcl_inhibitor: int, bcr-abl_inhibitor: int, benzodiazepine_receptor_agonist: int, beta_amyloid_inhibitor: int, bromodomain_inhibitor: int, btk_inhibitor: int, calcineurin_inhibitor: int, calcium_channel_blocker: int, cannabinoid_receptor_agonist: int, cannabinoid_receptor_antagonist: int, carbonic_anhydrase_inhibitor: int, casein_kinase_inhibitor: int, caspase_activator: int, catechol_o_methyltransferase_inhibitor: int, cc_chemokine_receptor_antagonist: int, cck_receptor_antagonist: int, cdk_inhibitor: int, chelating_agent: int, chk_inhibitor: int, chloride_channel_blocker: int, cholesterol_inhibitor: int, cholinergic_receptor_antagonist: int, coagulation_factor_inhibitor: int, corticosteroid_agonist: int, cyclooxygenase_inhibitor: int, cytochrome_p450_inhibitor: int, dihydrofolate_reductase_inhibitor: int, dipeptidyl_peptidase_inhibitor: int, diuretic: int, dna_alkylating_agent: int, dna_inhibitor: int, dopamine_receptor_agonist: int, dopamine_receptor_antagonist: int, egfr_inhibitor: int, elastase_inhibitor: int, erbb2_inhibitor: int, estrogen_receptor_agonist: int, estrogen_receptor_antagonist: int, faah_inhibitor: int, farnesyltransferase_inhibitor: int, fatty_acid_receptor_agonist: int, fgfr_inhibitor: int, flt3_inhibitor: int, focal_adhesion_kinase_inhibitor: int, free_radical_scavenger: int, fungal_squalene_epoxidase_inhibitor: int, gaba_receptor_agonist: int, gaba_receptor_antagonist: int, gamma_secretase_inhibitor: int, glucocorticoid_receptor_agonist: int, glutamate_inhibitor: int, glutamate_receptor_agonist: int, glutamate_receptor_antagonist: int, gonadotropin_receptor_agonist: int, gsk_inhibitor: int, hcv_inhibitor: int, hdac_inhibitor: int, histamine_receptor_agonist: int, histamine_receptor_antagonist: int, histone_lysine_demethylase_inhibitor: int, histone_lysine_methyltransferase_inhibitor: int, hiv_inhibitor: int, hmgcr_inhibitor: int, hsp_inhibitor: int, igf-1_inhibitor: int, ikk_inhibitor: int, imidazoline_receptor_agonist: int, immunosuppressant: int, insulin_secretagogue: int, insulin_sensitizer: int, integrin_inhibitor: int, jak_inhibitor: int, kit_inhibitor: int, laxative: int, leukotriene_inhibitor: int, leukotriene_receptor_antagonist: int, lipase_inhibitor: int, lipoxygenase_inhibitor: int, lxr_agonist: int, mdm_inhibitor: int, mek_inhibitor: int, membrane_integrity_inhibitor: int, mineralocorticoid_receptor_antagonist: int, monoacylglycerol_lipase_inhibitor: int, monoamine_oxidase_inhibitor: int, monopolar_spindle_1_kinase_inhibitor: int, mtor_inhibitor: int, mucolytic_agent: int, neuropeptide_receptor_antagon

In [0]:
train_df.printSchema()

root
-- sig_id: string (nullable = true)
-- cp_type: string (nullable = true)
-- cp_time: integer (nullable = true)
-- cp_dose: string (nullable = true)
-- g-0: double (nullable = true)
-- g-1: double (nullable = true)
-- g-2: double (nullable = true)
-- g-3: double (nullable = true)
-- g-4: double (nullable = true)
-- g-5: double (nullable = true)
-- g-6: double (nullable = true)
-- g-7: double (nullable = true)
-- g-8: double (nullable = true)
-- g-9: double (nullable = true)
-- g-10: double (nullable = true)
-- g-11: double (nullable = true)
-- g-12: double (nullable = true)
-- g-13: double (nullable = true)
-- g-14: double (nullable = true)
-- g-15: double (nullable = true)
-- g-16: double (nullable = true)
-- g-17: double (nullable = true)
-- g-18: double (nullable = true)
-- g-19: double (nullable = true)
-- g-20: double (nullable = true)
-- g-21: double (nullable = true)
-- g-22: double (nullable = true)
-- g-23: double (nullable = true)
-- g-24: double (nullable = true)
-- g-25: double (nullable = true)
-- g-26: double (nullable = true)
-- g-27: double (nullable = true)
-- g-28: double (nullable = true)
-- g-29: double (nullable = true)
-- g-30: double (nullable = true)
-- g-31: double (nullable = true)
-- g-32: double (nullable = true)
-- g-33: double (nullable = true)
-- g-34: double (nullable = true)
-- g-35: double (nullable = true)
-- g-36: double (nullable = true)
-- g-37: double (nullable = true)
-- g-38: double (nullable = true)
-- g-39: double (nullable = true)
-- g-40: double (nullable = true)
-- g-41: double (nullable = true)
-- g-42: double (nullable = true)
-- g-43: double (nullable = true)
-- g-44: double (nullable = true)
-- g-45: double (nullable = true)
-- g-46: double (nullable = true)
-- g-47: double (nullable = true)
-- g-48: double (nullable = true)
-- g-49: double (nullable = true)
-- g-50: double (nullable = true)
-- g-51: double (nullable = true)
-- g-52: double (nullable = true)
-- g-53: double (nullable = true)
-- g-54: double (nullable = true)
-- g-55: double (nullable = true)
-- g-56: double (nullable = true)
-- g-57: double (nullable = true)
-- g-58: double (nullable = true)
-- g-59: double (nullable = true)
-- g-60: double (nullable = true)
-- g-61: double (nullable = true)
-- g-62: double (nullable = true)
-- g-63: double (nullable = true)
-- g-64: double (nullable = true)
-- g-65: double (nullable = true)
-- g-66: double (nullable = true)
-- g-67: double (nullable = true)
-- g-68: double (nullable = true)
-- g-69: double (nullable = true)
-- g-70: double (nullable = true)
-- g-71: double (nullable = true)
-- g-72: double (nullable = true)
-- g-73: double (nullable = true)
-- g-74: double (nullable = true)
-- g-75: double (nullable = true)
-- g-76: double (nullable = true)
-- g-77: double (nullable = true)
-- g-78: double (nullable = true)
-- g-79: double (nullable = true)
-- g-80: double (nullable = true)
-- g-81: double (nullable = true)
-- g-82: double (nullable = true)
-- g-83: double (nullable = true)
-- g-84: double (nullable = true)
-- g-85: double (nullable = true)
-- g-86: double (nullable = true)
-- g-87: double (nullable = true)
-- g-88: double (nullable = true)
-- g-89: double (nullable = true)
-- g-90: double (nullable = true)
-- g-91: double (nullable = true)
-- g-92: double (nullable = true)
-- g-93: double (nullable = true)
-- g-94: double (nullable = true)
-- g-95: double (nullable = true)
-- g-96: double (nullable = true)
-- g-97: double (nullable = true)
-- g-98: double (nullable = true)
-- g-99: double (nullable = true)
-- g-100: double (nullable = true)
-- g-101: double (nullable = true)
-- g-102: double (nullable = true)
-- g-103: double (nullable = true)
-- g-104: double (nullable = true)
-- g-105: double (nullable = true)
-- g-106: double (nullable = true)
-- g-107: double (nullable = true)
-- g-108: double (nullable = true)
-- g-109: double (nullable = true)
-- g-110: double (nullable = true)
-- g-111: double (nullable = true)
-- g-112: double (nullable = true)
--

In [0]:
target_df.printSchema()

root
-- sig_id: string (nullable = true)
-- 5-alpha_reductase_inhibitor: integer (nullable = true)
-- 11-beta-hsd1_inhibitor: integer (nullable = true)
-- acat_inhibitor: integer (nullable = true)
-- acetylcholine_receptor_agonist: integer (nullable = true)
-- acetylcholine_receptor_antagonist: integer (nullable = true)
-- acetylcholinesterase_inhibitor: integer (nullable = true)
-- adenosine_receptor_agonist: integer (nullable = true)
-- adenosine_receptor_antagonist: integer (nullable = true)
-- adenylyl_cyclase_activator: integer (nullable = true)
-- adrenergic_receptor_agonist: integer (nullable = true)
-- adrenergic_receptor_antagonist: integer (nullable = true)
-- akt_inhibitor: integer (nullable = true)
-- aldehyde_dehydrogenase_inhibitor: integer (nullable = true)
-- alk_inhibitor: integer (nullable = true)
-- ampk_activator: integer (nullable = true)
-- analgesic: integer (nullable = true)
-- androgen_receptor_agonist: integer (nullable = true)
-- androgen_receptor_antagonist: integer (nullable = true)
-- anesthetic_-_local: integer (nullable = true)
-- angiogenesis_inhibitor: integer (nullable = true)
-- angiotensin_receptor_antagonist: integer (nullable = true)
-- anti-inflammatory: integer (nullable = true)
-- antiarrhythmic: integer (nullable = true)
-- antibiotic: integer (nullable = true)
-- anticonvulsant: integer (nullable = true)
-- antifungal: integer (nullable = true)
-- antihistamine: integer (nullable = true)
-- antimalarial: integer (nullable = true)
-- antioxidant: integer (nullable = true)
-- antiprotozoal: integer (nullable = true)
-- antiviral: integer (nullable = true)
-- apoptosis_stimulant: integer (nullable = true)
-- aromatase_inhibitor: integer (nullable = true)
-- atm_kinase_inhibitor: integer (nullable = true)
-- atp-sensitive_potassium_channel_antagonist: integer (nullable = true)
-- atp_synthase_inhibitor: integer (nullable = true)
-- atpase_inhibitor: integer (nullable = true)
-- atr_kinase_inhibitor: integer (nullable = true)
-- aurora_kinase_inhibitor: integer (nullable = true)
-- autotaxin_inhibitor: integer (nullable = true)
-- bacterial_30s_ribosomal_subunit_inhibitor: integer (nullable = true)
-- bacterial_50s_ribosomal_subunit_inhibitor: integer (nullable = true)
-- bacterial_antifolate: integer (nullable = true)
-- bacterial_cell_wall_synthesis_inhibitor: integer (nullable = true)
-- bacterial_dna_gyrase_inhibitor: integer (nullable = true)
-- bacterial_dna_inhibitor: integer (nullable = true)
-- bacterial_membrane_integrity_inhibitor: integer (nullable = true)
-- bcl_inhibitor: integer (nullable = true)
-- bcr-abl_inhibitor: integer (nullable = true)
-- benzodiazepine_receptor_agonist: integer (nullable = true)
-- beta_amyloid_inhibitor: integer (nullable = true)
-- bromodomain_inhibitor: integer (nullable = true)
-- btk_inhibitor: integer (nullable = true)
-- calcineurin_inhibitor: integer (nullable = true)
-- calcium_channel_blocker: integer (nullable = true)
-- cannabinoid_receptor_agonist: integer (nullable = true)
-- cannabinoid_receptor_antagonist: integer (nullable = true)
-- carbonic_anhydrase_inhibitor: integer (nullable = true)
-- casein_kinase_inhibitor: integer (nullable = true)
-- caspase_activator: integer (nullable = true)
-- catechol_o_methyltransferase_inhibitor: integer (nullable = true)
-- cc_chemokine_receptor_antagonist: integer (nullable = true)
-- cck_receptor_antagonist: integer (nullable = true)
-- cdk_inhibitor: integer (nullable = true)
-- chelating_agent: integer (nullable = true)
-- chk_inhibitor: integer (nullable = true)
-- chloride_channel_blocker: integer (nullable = true)
-- cholesterol_inhibitor: integer (nullable = true)
-- cholinergic_receptor_antagonist: integer (nullable = true)
-- coagulation_factor_inhibitor: integer (nullable = true)
-- corticosteroid_agonist: integer (nullable = true)
-- cyclooxygenase_inhibitor: integer (nullable = true)
-- cytochrome_p450_inhibitor: integer (nullable = true)
-- dihydrofolate_reductase_inhibitor: integer

In [0]:
def preproccess_features(df, cols, normalizer, excluded_col):
  
  cat_features = cols['cat_features']
  norm_features = cols['norm_features']
  normalizers = []
  
  indexed_cols = [''.join([col_name, '_indexed']) for col_name in cat_features]
  encoded_cols = [''.join([col_name, '_encoded']) for col_name in cat_features]
  vectorized_cols = [''.join([col_name, '_vectorized']) for col_name in norm_features]
  normalized_cols = [''.join([col_name, '_normalized']) for col_name in norm_features]
  vector_assemblers = [VectorAssembler(inputCols=[norm_features[i]], outputCol=vectorized_cols[i]) for i in range(len(norm_features))]
  string_indexers = [StringIndexer(inputCol=cat_features[i], outputCol=indexed_cols[i]) for i in range(len(cat_features))]

  for n in range(len(norm_features)):
    
    temp_normalizer = normalizer.copy()
    temp_normalizer.setInputCol(vectorized_cols[n])
    temp_normalizer.setOutputCol(normalized_cols[n])
    normalizers.append(temp_normalizer)
    
  encoder = OneHotEncoder(inputCols=indexed_cols, outputCols=encoded_cols)
  
  pipline = Pipeline(stages=string_indexers + vector_assemblers + normalizers + [encoder])
  
  prepro_df = pipline.fit(df).transform(df)
  prepro_df = prepro_df.drop(*indexed_cols + cat_features + vectorized_cols + norm_features)
  
  feature_assember = VectorAssembler(inputCols=[c for c in prepro_df.columns if c not in excluded_col], outputCol='features')
  prepro_df = feature_assember.transform(prepro_df).select(excluded_col + ['features'])
  
  return prepro_df

In [0]:
## preprocess train
target_cols = {'cat_features': ['cp_type', 'cp_dose'], 'norm_features': ['cp_time']}
normalizer = StandardScaler(withMean=True)
excluded_col = ['sig_id']
cleaned_train = preproccess_features(train_df, target_cols, normalizer, excluded_col)

## transform target
vectorizer = VectorAssembler(inputCols=[c for c in target_df.columns if c not in {'sig_id'}], outputCol='vectorized_target')
vectorized_target = vectorizer.transform(target_df).select(['sig_id', 'vectorized_target'])

## join two df
final_train = cleaned_train.join(vectorized_target.alias('target'), ['sig_id'])

In [0]:
## train test split
(train, validation) = final_train.randomSplit([0.8, 0.2], 16)

In [0]:
class EnsembleBaseClfs:

    def __init__(self, base_clf, num_classes, feature_col, target_col):

        self.base_clf = base_clf
        self._trained_clfs = []
        self.num_classes = num_classes
        self.feature_col = feature_col
        self.target_col = target_col

    def fit(self, train):

        for c in range(num_classes):
          
            print(c)
            temp_clf = self.base_clf.copy()
            temp_train = train.withColumn('new_target', vector_to_array(self.target_col)[c])
            
            temp_clf.setLabelCol('new_target')
            temp_clf.setFeaturesCol(self.feature_col)
            
            temp_clf = temp_clf.fit(temp_train)
            self._trained_clfs.append(temp_clf)

        return self

    def predict(self, x_test):

        ## TODO

In [0]:
## example with logistic Regression, first 5 classes

base_clf = LogisticRegression(maxIter=2)
num_classes = 5
feature_col = 'features'
target_col = 'vectorized_target'
clf = EnsembleBaseClfs(base_clf, num_classes, feature_col, target_col).fit(train)

0
1
2
3
4

Out[169]: DataFrame[sig_id: string, features: vector, vectorized_target: vector, rawPrediction: vector, probability: vector, prediction: double]